In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/naimul011/BanglaLicencePlateSuperResolutionGAN

Cloning into 'BanglaLicencePlateSuperResolutionGAN'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 64 (delta 24), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (64/64), done.


In [ ]:
!git clone https://github.com/zzxvictor/License-super-resolution

Cloning into 'License-super-resolution'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 53 (delta 8), reused 46 (delta 5), pack-reused 0
Unpacking objects: 100% (53/53), done.


In [ ]:
!mv License-super-resolution/Models ./Models

In [ ]:
!rm -r License-super-resolution

In [ ]:
!mv BanglaLicencePlateSuperResolutionGAN/Utilities/ ./Utilities

In [ ]:
!rm -r BanglaLicencePlateSuperResolutionGAN

In [ ]:
from Utilities.io import DataLoader
from Utilities.lossMetric import *
from Utilities.trainVal import MinMaxGame
from Models.RRDBNet import RRDBNet
from Models.GAN import Discriminator

from Utilities.io import DataLoader
from Utilities.lossMetric import *
from Utilities.trainVal import MinMaxGame
from Models.RRDBNet import RRDBNet
from Models.GAN import Discriminator

In [ ]:
#!cp -r /content/drive/MyDrive/Dataset/rawdata/banglalicense/*003.jpg /content/drive/MyDrive/Dataset/RSTrain

In [ ]:
#! mv /content/drive/MyDrive/Dataset/RSTrain/RSGAN/* /content/drive/MyDrive/Dataset/RSTrain

In [ ]:
! ls /content/drive/MyDrive/Processed_data/new_data | wc -l

308


### Load in the training dataset
I used the Chinese City Parking Dataset for this project. Please download the dataset from https://github.com/detectRecog/CCPD  
Before loading the dataset, it is critical that you run the preprocessing script (preprocess.py) first!!!   
` 
python preprocess.py 5 PATH_TO_UNZIPPED_DATA PATH_TO_OUTPUT_DIR
`

In [ ]:
import numpy as np
import glob
#PATH = 'PATH_TO_OUTPUT_DIR/192_96' # only use images with shape 192 by 96 for training
PATH = '/content/drive/MyDrive/Processed_data/training_data'
files = glob.glob(PATH + '/*.png') * 3  # data augmentation, same image with different brightness and contrast
np.random.shuffle(files)
train, val = files[:int(len(files)*0.8)], files[int(len(files)*0.8):]
loader = DataLoader()
trainData = DataLoader().load(train, batchSize=16)
valData = DataLoader().load(val, batchSize=64)

### Training

In [ ]:
discriminator = Discriminator()
extractor = buildExtractor()
generator = RRDBNet(blockNum=10)

80150528/80134624 [==============================] - 0s 0us/step


*  It's a good idea to pretrain the generator model before the min-max game - Reference: https://arxiv.org/abs/1701.00160

In [ ]:
# a simple custom loss function that combines MAE loss with VGG loss, as defined in the SRGAN paper
def contentLoss(y_true, y_pred):
    featurePred = extractor(y_pred)
    feature = extractor(y_true)
    mae = tf.reduce_mean(tfk.losses.mae(y_true, y_pred))
    return 0.1*tf.reduce_mean(tfk.losses.mse(featurePred, feature)) + mae

optimizer = tfk.optimizers.Adam(learning_rate=1e-3)
generator.compile(loss=contentLoss, optimizer=optimizer, metrics=[psnr, ssim])
# epoch is set to 1 for demonstration purpose. In practice I found 20 is a good number
# When the model reaches PSNR=20/ssim=0.65, we can start the min-max game
history = generator.fit(x=trainData, validation_data=valData, epochs=5, steps_per_epoch=300, validation_steps=100)

Epoch 1/5
300/300 [==============================] - 217s 645ms/step - loss: 1.6083 - psnr: 9.7582 - ssim: 0.3235 - val_loss: 1.1082 - val_psnr: 11.4960 - val_ssim: 0.3992
Epoch 2/5
300/300 [==============================] - 166s 554ms/step - loss: 0.9824 - psnr: 11.8665 - ssim: 0.4433 - val_loss: 0.8642 - val_psnr: 12.1781 - val_ssim: 0.4793
Epoch 3/5
300/300 [==============================] - 181s 603ms/step - loss: 0.7810 - psnr: 12.8913 - ssim: 0.5169 - val_loss: 0.7207 - val_psnr: 13.4001 - val_ssim: 0.5559
Epoch 4/5
300/300 [==============================] - 181s 603ms/step - loss: 0.6558 - psnr: 13.6367 - ssim: 0.5831 - val_loss: 0.6253 - val_psnr: 13.6495 - val_ssim: 0.5988
Epoch 5/5
300/300 [==============================] - 163s 546ms/step - loss: 0.5803 - psnr: 14.1881 - ssim: 0.6353 - val_loss: 0.5704 - val_psnr: 14.3725 - val_ssim: 0.6434


### Generative adverserial network training

In [ ]:
# training parameter. epoch is set to 1 for demonstration
# please train the network utill it reaches snRatio ~= 22 
PARAMS = dict(lrGenerator = 1e-4, 
              lrDiscriminator = 1e-4,
              epochs = 2, 
                stepsPerEpoch = 500, 
                valSteps = 100)
game = MinMaxGame(generator, discriminator, extractor)

# ideally peak signal noise ratio(snRation or psnr) should reach ~22

In [ ]:
log, valLog = game.train(trainData, valData, PARAMS)

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

### Save the model
Because I defined the model as inherited class of tf keras model, they cannot be safely serialized.  
Therefore, please save the weights only and follow the instructions in tutorial 1 to reload the model  
You can found my pretrained model in the *Pretrained* folder

In [ ]:
#generator.save_weights('/content/drive/MyDrive/Dataset/rrdb/', save_format='tf')
generator.save_weights( save_format='tf')
#generator.save_weights('generator.h5')

NameError: ignored

In [ ]:
!ls

In [ ]:
!cp rrdb.data-00000-of-00001 /content/drive/MyDrive/trained_model2

In [ ]:
!cp chechpoint /content/drive/MyDrive/trained_model2

In [ ]:
!cp rrdb.index /content/drive/MyDrive/trained_model2